In [1]:
datapath='../../features'
Flods=[0,1,2,3]
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import keras
from keras import optimizers
import matplotlib.pyplot as plt
from IPython import display
import time
import os
print(os.getcwd())

Using TensorFlow backend.


/home/mingchen/Course/CS235/CS235Project/scripts


In [46]:
class Network:

    def __init__(self, input_shape, num_classes, x_train, y_train,x_test,y_test):

        self.model = Sequential()

        self.input_shape = input_shape
        self.num_classes = num_classes

        self.x_train = x_train
        self.x_test=x_test
        self.y_train = []
        self.y_test=[]
        self.y_clean=[]
        for i in range(0, len(y_train)):
            self.y_train.append(y_train[i])
            self.y_clean.append(y_train[i])
        for i in range(0,len(y_test)):
            self.y_test.append(y_test[i])

        #self.y_train = keras.utils.to_categorical(self.y_train, self.num_classes)
        #self.y_test = keras.utils.to_categorical(self.y_test, self.num_classes)
        # self.y_clean = keras.utils.to_categorical(self.y_clean, self.num_classes)

        self.y_test=np.array(self.y_test)
        self.y_train=np.array(self.y_train)
        self.y_clean=np.array(self.y_clean)

        k=256
        self.model.add(Dense(k,activation='relu',input_shape=input_shape,
                             #kernel_initializer='he_normal',
                              kernel_initializer=keras.initializers.RandomNormal(0,0.001)))
        # self.model.add(Dense(k, activation='relu', input_shape=input_shape
        #                      , kernel_initializer=keras.initializers.RandomNormal(0, 1.0)))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(k/2,activation='relu',
                             kernel_initializer='he_normal'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(num_classes,activation='relu',
                             #kernel_initializer='he_normal'))
                             kernel_initializer=keras.initializers.RandomNormal(0,0.001/float(k))))
        #self.model.add(Dense(num_classes, activation='softmax'))

#         self.model.compile(loss='categorical_crossentropy',
#                            optimizer=optimizers.Adadelta(), metrics=["accuracy"])
        self.model.compile(loss='mean_squared_error',
                           optimizer=optimizers.Adadelta(), metrics=["accuracy"])
        #print self.model.summary()

    def set_weights(self, weight):
        self.model.set_weights(weight)

    def get_weights(self):
        return self.model.get_weights()

    def fit(self, batch_size, epochs,verbose=1,callbacks=[]):
        self.model.fit(
            self.x_train, self.y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=verbose,
            callbacks=callbacks,
            validation_data=(self.x_test,self.y_test),
            shuffle=True
        )


In [45]:
batch_size=128
epoch=200
num_classes=1
for flod in Flods:
    trainfile=open('{0}/cv_{1}_4_{2}_4/cv_train_features_{1}_4_{2}_4.csv'.format(datapath,flod,flod+1))
    testfile=open('{0}/cv_{1}_4_{2}_4/cv_test_features_{1}_4_{2}_4.csv'.format(datapath,flod,flod+1))
    trainX=[]
    trainY=[]
    testX=[]
    testY=[]
    for line in trainfile:
        line=line.split(',')[1:]
        if line[0]=='safe_type':
            continue
        line=[float(k) for k in line]
        trainX.append(line[1:])
        trainY.append(line[0])
    trainX=np.array(trainX)[:,:]
    meanX=np.mean(trainX)
    trainX-=meanX
    trainY=np.array(trainY).astype(int)
    #trainY[trainY==0]=-1
    print(trainX.shape,trainY.shape)
    
    for line in testfile:
        line=line.split(',')[1:]
        if line[0]=='safe_type':
            continue
        line=[float(k) for k in line]
        testX.append(line[1:])
        testY.append(line[0])
    testX=np.array(testX)[:,:]
    testX-=meanX
    testY=np.array(testY).astype(int)
    #testY[testY==0]=-1
    print(testX.shape,testY.shape)
    input_shape=(trainX.shape[1],)
    net = Network(input_shape, num_classes, trainX, trainY, testX, testY)
    #print(net.model.summary())
    net.fit(batch_size,epoch,verbose=1)
    print(net.model.evaluate(testX,texsY))
#     err=np.sum(testY[:,0]*ans<=0)/testY.shape[0]
#     print(err)

((18000, 55), (18000,))
((6000, 55), (6000,))
Train on 18000 samples, validate on 6000 samples
Epoch 1/200
18000/18000 [==============================] - 2s 110us/step - loss: 0.1619 - acc: 0.7956 - val_loss: 0.1176 - val_acc: 0.8483
Epoch 2/200
18000/18000 [==============================] - 1s 28us/step - loss: 0.1351 - acc: 0.8204 - val_loss: 0.1274 - val_acc: 0.8357
Epoch 3/200
18000/18000 [==============================] - 1s 29us/step - loss: 0.1243 - acc: 0.8404 - val_loss: 0.1225 - val_acc: 0.8560
Epoch 4/200
18000/18000 [==============================] - 1s 28us/step - loss: 0.1179 - acc: 0.8542 - val_loss: 0.1054 - val_acc: 0.8690
Epoch 5/200
18000/18000 [==============================] - 1s 28us/step - loss: 0.1125 - acc: 0.8633 - val_loss: 0.1109 - val_acc: 0.8745
Epoch 6/200
18000/18000 [==============================] - 1s 29us/step - loss: 0.1101 - acc: 0.8673 - val_loss: 0.1044 - val_acc: 0.8732
Epoch 7/200
18000/18000 [==============================] - 1s 29us/step - lo

KeyboardInterrupt: 

In [47]:
batch_size=128
epoch=300
trainfile=open('{0}/full/train_features_full.csv'.format(datapath,flod,flod+1))
testfile=open('{0}/full/test_features_full.csv'.format(datapath,flod,flod+1))
trainX=[]
trainY=[]
testX=[]
testY=[]
for line in trainfile:
    line=line.split(',')[1:]
    if line[0]=='safe_type':
        continue
    line=[float(k) for k in line]
    trainX.append(line[1:])
    trainY.append(line[0])
trainX=np.array(trainX)
trainY=np.array(trainY)[:,np.newaxis].astype(int)
print(trainX.shape,trainY.shape)

for line in testfile:
    line=line.split(',')[1:]
    if line[0]=='safe_type':
        continue
    line=[float(k) for k in line]
    testX.append(line[1:])
    testY.append(line[0])
testX=np.array(testX)
testY=np.array(testY)[:,np.newaxis].astype(int)
print(testX.shape,testY.shape)

input_shape=(trainX.shape[1],)
net = Network(input_shape, num_classes, trainX, trainY, testX, testY)
net.fit(batch_size,epoch,verbose=1)
print(net.model.evaluate(testX,testY))


((24000, 55), (24000, 1))
((6000, 55), (6000, 1))
Train on 24000 samples, validate on 6000 samples
Epoch 1/300
24000/24000 [==============================] - 2s 88us/step - loss: 0.1700 - acc: 0.8118 - val_loss: 0.1367 - val_acc: 0.8155
Epoch 2/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.1334 - acc: 0.8265 - val_loss: 0.1426 - val_acc: 0.8135
Epoch 3/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.1270 - acc: 0.8390 - val_loss: 0.1547 - val_acc: 0.7775
Epoch 4/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.1244 - acc: 0.8432 - val_loss: 0.1433 - val_acc: 0.8042
Epoch 5/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.1197 - acc: 0.8452 - val_loss: 0.1211 - val_acc: 0.8505
Epoch 6/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.1171 - acc: 0.8501 - val_loss: 0.1156 - val_acc: 0.8583
Epoch 7/300
24000/24000 [==============================] - 1s 29us/step -

24000/24000 [==============================] - 1s 27us/step - loss: 0.0688 - acc: 0.9121 - val_loss: 0.1075 - val_acc: 0.8532
Epoch 60/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0676 - acc: 0.9143 - val_loss: 0.1113 - val_acc: 0.8485
Epoch 61/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0680 - acc: 0.9156 - val_loss: 0.1067 - val_acc: 0.8650
Epoch 62/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0684 - acc: 0.9142 - val_loss: 0.1022 - val_acc: 0.8652
Epoch 63/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0691 - acc: 0.9135 - val_loss: 0.1036 - val_acc: 0.8653
Epoch 64/300
24000/24000 [==============================] - 1s 29us/step - loss: 0.0682 - acc: 0.9144 - val_loss: 0.1041 - val_acc: 0.8633
Epoch 65/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0683 - acc: 0.9147 - val_loss: 0.1042 - val_acc: 0.8617
Epoch 66/300
24000/24000 [==============

Epoch 118/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0627 - acc: 0.9210 - val_loss: 0.1118 - val_acc: 0.8530
Epoch 119/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0617 - acc: 0.9215 - val_loss: 0.1169 - val_acc: 0.8410
Epoch 120/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0616 - acc: 0.9227 - val_loss: 0.1058 - val_acc: 0.8622
Epoch 121/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0636 - acc: 0.9200 - val_loss: 0.1067 - val_acc: 0.8608
Epoch 122/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0617 - acc: 0.9231 - val_loss: 0.1118 - val_acc: 0.8480
Epoch 123/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0615 - acc: 0.9227 - val_loss: 0.1604 - val_acc: 0.7930
Epoch 124/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0635 - acc: 0.9203 - val_loss: 0.1361 - val_acc: 0.8123
Epoch 125/300
24000/

24000/24000 [==============================] - 1s 27us/step - loss: 0.0614 - acc: 0.9215 - val_loss: 0.1204 - val_acc: 0.8297
Epoch 177/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0594 - acc: 0.9255 - val_loss: 0.1276 - val_acc: 0.8237
Epoch 178/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0588 - acc: 0.9274 - val_loss: 0.1238 - val_acc: 0.8350
Epoch 179/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0598 - acc: 0.9255 - val_loss: 0.1132 - val_acc: 0.8475
Epoch 180/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0598 - acc: 0.9255 - val_loss: 0.1169 - val_acc: 0.8395
Epoch 181/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0584 - acc: 0.9265 - val_loss: 0.1163 - val_acc: 0.8408
Epoch 182/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0594 - acc: 0.9265 - val_loss: 0.1284 - val_acc: 0.8212
Epoch 183/300
24000/24000 [=======

24000/24000 [==============================] - 1s 28us/step - loss: 0.0560 - acc: 0.9318 - val_loss: 0.1293 - val_acc: 0.8180
Epoch 235/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0580 - acc: 0.9273 - val_loss: 0.1172 - val_acc: 0.8415
Epoch 236/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0547 - acc: 0.9319 - val_loss: 0.1321 - val_acc: 0.8148
Epoch 237/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0587 - acc: 0.9274 - val_loss: 0.1131 - val_acc: 0.8547
Epoch 238/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0557 - acc: 0.9315 - val_loss: 0.1231 - val_acc: 0.8323
Epoch 239/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0576 - acc: 0.9281 - val_loss: 0.1232 - val_acc: 0.8370
Epoch 240/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0570 - acc: 0.9298 - val_loss: 0.1317 - val_acc: 0.8190
Epoch 241/300
24000/24000 [=======

24000/24000 [==============================] - 1s 27us/step - loss: 0.0574 - acc: 0.9301 - val_loss: 0.1231 - val_acc: 0.8353
Epoch 293/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0549 - acc: 0.9318 - val_loss: 0.1225 - val_acc: 0.8383
Epoch 294/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0550 - acc: 0.9311 - val_loss: 0.1172 - val_acc: 0.8450
Epoch 295/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0568 - acc: 0.9304 - val_loss: 0.1296 - val_acc: 0.8228
Epoch 296/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0567 - acc: 0.9298 - val_loss: 0.1197 - val_acc: 0.8430
Epoch 297/300
24000/24000 [==============================] - 1s 28us/step - loss: 0.0553 - acc: 0.9298 - val_loss: 0.1197 - val_acc: 0.8405
Epoch 298/300
24000/24000 [==============================] - 1s 27us/step - loss: 0.0562 - acc: 0.9296 - val_loss: 0.1269 - val_acc: 0.8282
Epoch 299/300
24000/24000 [=======